In [1]:
import os
import numpy as np
import copy
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from marvin.tools import Maps
from marvin import config
import marvin.utils.plot.map as mapplot
from marvin.tools.image import Image

/home/idies/miniconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/idies/miniconda3/lib/python3.7/site-packages/marvin/core/exceptions.py:51: UserWarning: cannot initiate Sentry error reporting: [Errno 6] No such device or address.
  UserWarning)


INFO: No release version set. Setting default to DR15


/home/idies/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/idies/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/idies/miniconda3/lib/python3.7/site-packages/pympler/asizeof.py:1491: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  numpy.matrix(range(0)), numpy.ma.masked_array([])):
/home/idies/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/idies/minico

In [2]:
# These functions come from the manga_flipping_funct.py file
from manga_flipping_funct import snr_mask, flip_map, disk

In [3]:
# set the cosmology for converting between angular size to physical size at a given redshift
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)

In [4]:
targets = np.loadtxt("/home/idies/workspace/Storage/adiamond/persistent/bgl_manga/plateifu_csv_files/jones2017.csv", delimiter=',', dtype=str)
targets = targets[1:]

In [5]:
filename = 'ad_jones.pdf'

In [7]:
with PdfPages(filename) as pdf:
    for i in range(0,len(targets)):
        print(str(targets[i]))
        try:
            maps = Maps(plateifu=targets[i], bintype='HYB10', template='GAU-MILESHC')
            snr_minimum = 3.

            # Creating flux maps
            ha = maps.emline_gflux_ha_6564
            mask_ha_flux = snr_mask(ha, snr_minimum)

            # [nII]/Ha
            nii = maps.emline_gflux_nii_6585
            nii_ha = nii/ha
            mask_nii_ha = mask_ha_flux | snr_mask(nii, snr_minimum)

            # [oiii]/Hb
            hb = maps.emline_gflux_hb_4862 
            oiii_5008 = maps.emline_gflux_oiii_5008
            oiii_hb = oiii_5008/hb
            mask_oiii_hb = snr_mask(hb, snr_minimum) | snr_mask(oiii_5008, snr_minimum)


            # Ha vel dispersion, velocity, stellar vel, 
            ha_sig = maps.emline_gsigma_ha_6564
            ha_vel = maps.emline_gvel_ha_6564
            stvel = maps.stellar_vel


            # Ha velocity - stellar velocity map
            diff = copy.deepcopy(stvel)
            diff.value[:,:] = ha_vel.value - stvel.value
            diff.ivar = 1. / ((1. / ha_vel.ivar) + (1. / stvel.ivar))
            diff.mask = ha_vel.mask | stvel.mask | mask_ha_flux
            diff.property_name = 'Ha vel - stellar vel'

            # Flipped Maps 
            flipped_vel, flip_flux_mask, asymmetry, major_axis = flip_map(maps, ha_vel, ha, snr_minimum)


            #Plotting
            fig = plt.figure(constrained_layout=False, figsize=(25,20))
            spec = gridspec.GridSpec(ncols=4, nrows=3, figure=fig,  hspace=0.2)

            #(0,0) IMAGE 
            ax1 = fig.add_subplot(spec[0,0])
            im = Image(targets[i], download=False)
            ax1.imshow(im.data)
        #         plt.grid(None)
            ax1.set_title(str(targets[i]))

            #(0,1) FLUX 
            ax2 = fig.add_subplot(spec[0,1])
            flux = Maps(plateifu=targets[i]).emline_gflux_ha_6564
            flux.plot(fig=fig, ax=ax2, snr_min=0)

            #(0,2) 1 re size disk cutout
            ax3 = fig.add_subplot(spec[0,2])
            ha_vel_copy = copy.deepcopy(ha_vel)
            ha_vel_copy[disk(maps, ha_vel, disk_re_size=1.)] = 0 
            ha_vel_copy.plot(fig=fig, ax=ax3, mask=mask_ha_flux, title=r'$H_\alpha with disk= 1.0 re$')

            #(0,3) 2 re size disk cutout
            ax4 = fig.add_subplot(spec[0,3])
            ha_vel_copy[disk(maps, ha_vel, disk_re_size=1.5)] = 0
            ha_vel_copy.plot(fig=fig, ax=ax4, mask=mask_ha_flux, title=r'$H_\alpha with disk= 1.5 re$')


            #(1,0) Hα VELOCITY
            ax5 = fig.add_subplot(spec[1, 0])
            ha_vel[np.where(major_axis)] = 0
            ha_vel.plot(fig=fig, ax=ax5, mask=mask_ha_flux, title=r'$H_\alpha$')


            #(1,1) Hα FLIPPED VELOCITY
            ax6 = fig.add_subplot(spec[1, 1])
            flipped_vel.plot(fig=fig, ax=ax6, title='Ha vel 6564 Flip', mask=flip_flux_mask)

            #(1,2) ASYMMETRY 
            ax7 = fig.add_subplot(spec[1, 2])
            asymmetry.plot(fig=fig, ax=ax7, title='Ha vel 6564 Asymmetry',mask=asymmetry.mask)

            #(1,3) VELOCITY DISPERSION
            ax8 = fig.add_subplot(spec[1, 3])
            ha_sig.plot(fig=fig, ax=ax8, mask=mask_ha_flux)

            #(2,0) STELLAR VELOCITY
            ax9 = fig.add_subplot(spec[2, 0])
            stvel.plot(fig=fig, ax=ax9, mask=mask_ha_flux)

            #(2,1) VELOCITY - STELLAR VELOCITY
            ax10 = fig.add_subplot(spec[2, 1])
            diff.plot(fig=fig, ax=ax10, title='Ha vel - stellar vel')

            #(2,2) Nii/Ha
            ax11 = fig.add_subplot(spec[2, 2])
            nii_ha.plot(fig=fig, ax=ax11, title='[NII]/Ha', mask=mask_nii_ha)

            #(2,3) Oiii/Hb
            ax12 = fig.add_subplot(spec[2, 3])
            oiii_hb.plot(fig=fig, ax=ax12, title='[oiii 5008]/Hb',  mask=mask_oiii_hb)


            fig.tight_layout()

            pdf.savefig(facecolor=fig.get_facecolor(), edgecolor='none')
            plt.close()

        except:
            print('error')
            pass
#os.system("open %s &" % filename)

8083-3703


/home/idies/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in true_divide


8137-12701
8135-12705
8137-3704
8137-12705
8138-6103 
error
8140-12702
8143-12703 
error
8249-12701
8247-12705 
error
8250-12705 
error
8252-9101 
error
8253-12704 
error
8254-12702 
error
8256-12701 
error
8448-6102 
error
8257-9101 
error
8257-12702 
error
8451-12705 
error
8448-12704
8257-12705 
error
8466-12703 
error
8259-12702 
error
8259-12703 
error
8263-12702
8341-12701 
error
8317-6101 
error
8465-9101 
error
8330-12704 
error
7495-3703 
error
8325-3702 
error
8329-12703 
error
8335-12703 
error
8552-12702
7443-12704 
error
8551-12704 
error
8485-3704 
error
8481-12702
8486-12704 
error
8603-6103 
error
8484-6101 
error
7991-6103 
error
7962-12705 
error
7990-9101 
error
7815-6101 
error
8618-12701 
error
7975-3701 
error
7977-12704
8552-6102
8483-9102
8606-6101
8618-9102
    
error
  
error
  
error
  
error
  
error
10220-6104
error
7960-12702
7960-6103
7979-12701
error
8077-9101
8081-1901
8083-3703
8085-3701
8086-3704
8137-3704
8138-6101
8138-6103
8141-3701
8144-12705
8249